In [1]:
# import some shit
import numpy as np
import pandas as pd
import random
import tensorflow as tf

In [2]:
# read json & drop column
def to_df(file_path):
    with open(file_path, "r")as f:
        i, df = 0, {}
        for line in f:
            df[i] = eval(line)
            i += 1
        df = pd.DataFrame.from_dict(df, orient='index')
        return df
review_df = to_df('Deep-Interest-Network/reviews_Electronics_5.json')[['reviewerID', 'asin', 'unixReviewTime']]
item_df = to_df('Deep-Interest-Network/meta_Electronics.json')[['asin', 'categories']]

In [3]:
# fine-grain categories
item_df['categories'] = item_df['categories'].map(lambda x: x[-1][-1])

In [4]:
# items in review_df
item_df = item_df[item_df['asin'].isin(review_df['asin'].unique())]
item_df = item_df.reset_index(drop=True)

In [5]:
# map id to integer
def build_map(df, column):
    key = sorted(df[column].unique().tolist())
    map_rule = dict(zip(key, range(0, len(df))))
    df[column] = df[column].map(lambda x: map_rule[x])
    return map_rule, key
map_rule_reviewerID, key_reviewerID= build_map(review_df, 'reviewerID')
map_rule_asin, key_asin= build_map(item_df, 'asin')
map_rule_categories, key_categories = build_map(item_df, 'categories')
review_df['asin'] = review_df['asin'].map(lambda x: map_rule_asin[x])

In [6]:
# count some shit
user_count, item_count, cate_count, example_count = \
len(map_rule_reviewerID),len(map_rule_asin),len(map_rule_categories),review_df.shape[0]

In [7]:
# categories ready
item_df = item_df.sort_values(['asin']).reset_index(drop=True)
cate_list = np.array(item_df['categories'], dtype=np.int32)
print(item_df.head())

   asin  categories
0     0         217
1     1          47
2     2         167
3     3         206
4     4         210


In [8]:
# reviews ready
review_df = review_df.sort_values(by=['reviewerID', 'unixReviewTime']).reset_index(drop=True)
print(review_df.head())

   reviewerID  asin  unixReviewTime
0           0    75      1385337600
1           1   890      1358035200
2           2   643      1361750400
3           3   168      1390003200
4           4   533      1350086400


In [9]:
# build trainset and testset
train_set, test_set = [], []
for reviewerID, history in review_df.groupby('reviewerID'):
    # define positive samples
    positive_sample = history['asin'].tolist()
    
    # define negative samples
    def negative_sampling():
        negative_sample = positive_sample[0]
        while negative_sample in positive_sample:
            negative_sample = random.randint(0, len(item_df)-1)
        return negative_sample
    negative_sample = [negative_sampling() for i in range(len(positive_sample))]
    
    # build training set & testing set
    for i in range(1, len(positive_sample)):
        history = positive_sample[:i]
        if i == len(positive_sample)-1:
            label = (positive_sample[i], negative_sample[i])
            test_set.append((reviewerID, history, label))
        else:
            train_set.append((reviewerID, history, positive_sample[i], 1))
            train_set.append((reviewerID, history, negative_sample[i], 0))
# shuffle the deck
random.shuffle(train_set)
random.shuffle(test_set)

In [10]:
print(len(train_set))
print(len(test_set))

3988
3025


In [11]:
# transform data records to data batches
class InputBatchData(object):
    
    # define data, batch_size, epoch_size, idx
    def __init__(self, data, batch_size):
        self.data = data
        self.batch_size = batch_size
        self.epoch_size = len(self.data) // self.batch_size
        if self.epoch_size * self.batch_size < len(self.data):
            self.epoch_size += 1
        else:
            self.epoch_size
        self.idx = 0
        
    # start iteration
    def __iter__(self):
        return self
    
    # next batch
    def __next__(self):
        
        # conditions to stop iteration
        if self.idx == self.epoch_size:
            raise StopIteration
            
        # data records to tranform
        start, end = self.idx * self.batch_size, min((self.idx + 1) * self.batch_size, len(self.data))
        batch_data = self.data[start : end]
        self.idx += 1
        
        # user_id, item_id, y, sample_len in batch
        user_id, item_id, y, sample_len = [], [], [], []
        for i in batch_data:
            user_id.append(i[0])
            item_id.append(i[2])
            y.append(i[3])
            sample_len.append(len(i[1]))
            
        # hist_i (user-item metrix) in batch
        hist_i = np.zeros([len(batch_data), max(sample_len)], np.int64)
        k = 0
        for l in batch_data:
            for j in range(len(l[1])):
                hist_i[k][j] = l[1][j]
            k += 1
        
        return self.idx, (user_id, item_id, y, hist_i, sample_len)